In [2]:
import os, math, json, time, pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
def load_and_prepare_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = (x_train / 255.0).astype("float32")
    x_test  = (x_test  / 255.0).astype("float32")

    x_train_mlp = x_train
    x_test_mlp  = x_test

    x_train_cnn = np.expand_dims(x_train, axis=-1)
    x_test_cnn  = np.expand_dims(x_test , axis=-1)

    print(f"[Shapes] x_train_mlp: {x_train_mlp.shape}, x_test_mlp: {x_test_mlp.shape}")
    print(f"[Shapes] x_train_cnn: {x_train_cnn.shape}, x_test_cnn: {x_test_cnn.shape}")

    return (x_train_mlp, y_train, x_test_mlp, y_test,
            x_train_cnn, y_train, x_test_cnn, y_test)

In [4]:
def build_mlp():
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ], name="MLP_FashionMNIST")

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()
    return model

In [5]:
def build_cnn():
    model = keras.Sequential([
        layers.Conv2D(16, 3, activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D(2),
        layers.Conv2D(32, 3, activation="relu"),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ], name="CNN_FashionMNIST")

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()
    return model

In [6]:
def train_and_eval(model, x_train, y_train, x_test, y_test, epochs=5, batch_size=64):
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=2
    )
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return history, test_loss, test_acc

In [7]:
def count_trainable_params(model):
    return int(np.sum([np.prod(v.shape) for v in model.trainable_weights]))

def save_and_size(model, path):
    model.save(path)
    size_mb = os.path.getsize(path) / (1024**2)
    return size_mb

In [8]:
def estimate_layer_flops(layer, input_shape):
    flops = 0

    if isinstance(layer, layers.Dense):
        in_dim = np.prod(input_shape[1:])
        out_dim = layer.units
        flops = 2 * in_dim * out_dim + out_dim
        out_shape = (input_shape[0], out_dim)

    elif isinstance(layer, layers.Conv2D):

        strides = layer.strides
        padding = layer.padding
        kernel_h, kernel_w = layer.kernel_size
        filters = layer.filters


        _, Hin, Win, Cin = input_shape


        if padding == "valid":
            Hout = (Hin - kernel_h) // strides[0] + 1
            Wout = (Win - kernel_w) // strides[1] + 1
        else:
            Hout = math.ceil(Hin / strides[0])
            Wout = math.ceil(Win / strides[1])

        flops = 2 * Hout * Wout * filters * (kernel_h * kernel_w * Cin) + (Hout * Wout * filters)
        out_shape = (input_shape[0], Hout, Wout, filters)

    elif isinstance(layer, layers.MaxPooling2D):
        pool_size = layer.pool_size
        strides = layer.strides or pool_size
        padding = layer.padding

        _, Hin, Win, Cin = input_shape
        if padding == "valid":
            Hout = (Hin - pool_size[0]) // strides[0] + 1
            Wout = (Win - pool_size[1]) // strides[1] + 1
        else:
            Hout = math.ceil(Hin / strides[0])
            Wout = math.ceil(Win / strides[1])
        out_shape = (input_shape[0], Hout, Wout, Cin)

    elif isinstance(layer, layers.Flatten):
        out_shape = (input_shape[0], int(np.prod(input_shape[1:])))
    else:
        out_shape = input_shape

    return flops, out_shape


In [9]:
def estimate_model_flops_inference(model, input_spec):
    total_flops = 0
    current_shape = input_spec
    for layer in model.layers:
        f, current_shape = estimate_layer_flops(layer, current_shape)
        total_flops += f
    return int(total_flops)

In [10]:
def estimate_training_flops_from_inference(flops_inference):
    return int(3 * flops_inference)

In [11]:
def estimate_training_memory_bytes(model, optimizer="adam"):

    params = count_trainable_params(model)
    bytes_per_param = 4
    if optimizer.lower() == "adam":
        factor = 4
    else:
        factor = 2
    return params * bytes_per_param * factor

def to_mb(x_bytes):
    return x_bytes / (1024**2)

def to_gb(x_bytes):
    return x_bytes / (1024**3)


In [12]:
def main():
    out_dir = pathlib.Path("artifacts_fmnist")
    out_dir.mkdir(exist_ok=True)

    (x_train_mlp, y_train, x_test_mlp, y_test,
     x_train_cnn, _, x_test_cnn, _) = load_and_prepare_data()

    print("\n=== Build MLP ===")
    mlp_model = build_mlp()
    print("\n=== Build CNN ===")
    cnn_model = build_cnn()

    print("\n=== Train MLP (5 epochs, bs=64) ===")
    mlp_hist, mlp_test_loss, mlp_test_acc = train_and_eval(
        mlp_model, x_train_mlp, y_train, x_test_mlp, y_test, epochs=5, batch_size=64
    )
    print("\n=== Train CNN (5 epochs, bs=64) ===")
    cnn_hist, cnn_test_loss, cnn_test_acc = train_and_eval(
        cnn_model, x_train_cnn, y_train, x_test_cnn, y_test, epochs=5, batch_size=64
    )

    mlp_path = str(out_dir / "mlp_model.h5")
    cnn_path = str(out_dir / "cnn_model.h5")
    mlp_size_mb = save_and_size(mlp_model, mlp_path)
    cnn_size_mb = save_and_size(cnn_model, cnn_path)

    mlp_params = count_trainable_params(mlp_model)
    cnn_params = count_trainable_params(cnn_model)

    mlp_infer_flops = estimate_model_flops_inference(mlp_model, input_spec=(1, 28, 28))
    cnn_infer_flops = estimate_model_flops_inference(cnn_model, input_spec=(1, 28, 28, 1))

    mlp_train_flops = estimate_training_flops_from_inference(mlp_infer_flops)
    cnn_train_flops = estimate_training_flops_from_inference(cnn_infer_flops)

    mlp_train_mem_bytes = estimate_training_memory_bytes(mlp_model, optimizer="adam")
    cnn_train_mem_bytes = estimate_training_memory_bytes(cnn_model, optimizer="adam")

    table = [
        {
            "Model": "MLP",
            "Test Accuracy": round(float(mlp_test_acc), 4),
            "Trainable Parameters": mlp_params,
            "Saved Model Size (MB)": round(mlp_size_mb, 3),
            "FLOPs (Training)": mlp_train_flops,
            "FLOPs (Inference)": mlp_infer_flops,
            "Training Memory (MB)": round(to_mb(mlp_train_mem_bytes), 3),
        },
        {
            "Model": "CNN",
            "Test Accuracy": round(float(cnn_test_acc), 4),
            "Trainable Parameters": cnn_params,
            "Saved Model Size (MB)": round(cnn_size_mb, 3),
            "FLOPs (Training)": cnn_train_flops,
            "FLOPs (Inference)": cnn_infer_flops,
            "Training Memory (MB)": round(to_mb(cnn_train_mem_bytes), 3),
        },
    ]

    print("\n==================== RESULTS TABLE ====================")
    headers = list(table[0].keys())
    print("\t".join(headers))
    for row in table:
        print("\t".join(str(row[h]) for h in headers))

    results = {
        "mlp": {
            "test_acc": float(mlp_test_acc),
            "test_loss": float(mlp_test_loss),
            "params": mlp_params,
            "size_mb": float(round(mlp_size_mb, 3)),
            "flops_train": mlp_train_flops,
            "flops_infer": mlp_infer_flops,
            "train_mem_mb": float(round(to_mb(mlp_train_mem_bytes), 3)),
        },
        "cnn": {
            "test_acc": float(cnn_test_acc),
            "test_loss": float(cnn_test_loss),
            "params": cnn_params,
            "size_mb": float(round(cnn_size_mb, 3)),
            "flops_train": cnn_train_flops,
            "flops_infer": cnn_infer_flops,
            "train_mem_mb": float(round(to_mb(cnn_train_mem_bytes), 3)),
        }
    }
    with open(out_dir / "results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


    higher_acc = "CNN" if results["cnn"]["test_acc"] >= results["mlp"]["test_acc"] else "MLP"
    smaller_params = "CNN" if results["cnn"]["params"] <= results["mlp"]["params"] else "MLP"
    print(f"- The highest resolution model: {higher_acc}")
    print(f"- The model has fewer parameters: {smaller_params}")

if __name__ == "__main__":
    main()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[Shapes] x_train_mlp: (60000, 28, 28), x_test_mlp: (10000, 28, 28)
[Shapes] x_train_cnn: (60000, 28, 28, 1), x_test_cnn: (10000, 28, 28, 1)

=== Build MLP ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "MLP_FashionMNIST"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

...

 Total params: 56,714 (221.54 KB)

 Trainable params: 56,714 (221.54 KB)

 Non-trainable params: 0 (0.00 B)


=== Train MLP (5 epochs, bs=64) ===
Epoch 1/5
844/844 - 8s - 10ms/step - accuracy: 0.8251 - loss: 0.4926 - val_accuracy: 0.8588 - val_loss: 0.3883
Epoch 2/5
844/844 - 5s - 6ms/step - accuracy: 0.8657 - loss: 0.3672 - val_accuracy: 0.8745 - val_loss: 0.3382
Epoch 3/5
844/844 - 10s - 12ms/step - accuracy: 0.8802 - loss: 0.3278 - val_accuracy: 0.8787 - val_loss: 0.3284
Epoch 4/5
844/844 - 5s - 6ms/step - accuracy: 0.8868 - loss: 0.3052 - val_accuracy: 0.8732 - val_loss: 0.3452
Epoch 5/5
844/844 - 6s - 8ms/step - accuracy: 0.8934 - loss: 0.2875 - val_accuracy: 0.8823 - val_loss: 0.3233

=== Train CNN (5 epochs, bs=64) ===
Epoch 1/5
844/844 - 28s - 33ms/step - accuracy: 0.8004 - loss: 0.5584 - val_accuracy: 0.8480 - val_loss: 0.4254
Epoch 2/5
844/844 - 39s - 46ms/step - accuracy: 0.8665 - loss: 0.3689 - val_accuracy: 0.8785 - val_loss: 0.3355
Epoch 3/5
844/844 - 41s - 48ms/step - accuracy: 0.8829 - loss: 0.3254 - val_accuracy: 0.8827 - val_loss: 0.3260
Epoch 4/5
844/844 - 42s - 49ms/step -


==================== RESULTS TABLE ====================
Model	Test Accuracy	Trainable Parameters	Saved Model Size (MB)	FLOPs (Training)	FLOPs (Inference)	Training Memory (MB)
MLP	0.8742	235146	2.722	1409694	469898	3.588
CNN	0.8898	56714	0.689	4284798	1428266	0.865
- The highest resolution model: CNN
- The model has fewer parameters: CNN


| Model   | Test Accuracy | Params  | Size (MB) | FLOPs (Train) | FLOPs (Inference) | Training Memory |
| ------- | ------------- | ------- | --------- | ------------- | ----------------- | --------------- |
| **MLP** | 0.8742        | 235,146 | 2.722     | 1,409,694     | 469,898           | 3.588 MB        |
| **CNN** | 0.8898        | 56,714  | 0.689     | 4,284,798     | 1,428,266         | 0.865 MB        |


==================== CONCLUSION ====================
- النموذج ذو الدقة الأعلى غالبًا: CNN
- النموذج ذو عدد براميترات أقل: CNN
- التوازن/التبادل: عادة تتفوّق CNN على الصور لأنها تستغل محليّة البيكسلات ومرشّحات الالتفاف والتشارك في الأوزان، بينما الـMLP أبسط وأصغر (معلمات أقل) لكنه يهمل البُنى المكانية في الصور.